In [4]:
# imports
import pandas as pd
import numpy as np
import geopy as gp
import datetime
import os
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [5]:
# creating a dataset list for all datasets
#path = '../datasets/data_for_mvp'
files = [file for file in os.listdir('gtfs_2')]

In [6]:
# Loading each dataset to each dataframe
agency = pd.read_csv('gtfs_2/agency.txt')
dates = pd.read_csv('gtfs_2/calendar_dates.txt')
stop_times = pd.read_csv('gtfs_2/stop_times.txt')
frequencies = pd.read_csv('gtfs_2/frequencies.txt')
shapes = pd.read_csv('gtfs_2/shapes.txt')
trips = pd.read_csv('gtfs_2/trips.txt')
stops = pd.read_csv('gtfs_2/stops.txt')
calendar = pd.read_csv('gtfs_2/calendar.txt')
routes = pd.read_csv('gtfs_2/routes.txt')

In [9]:
# Creating a dict w/all datasets:
all_ds = {"agency": agency, 
          "dates": dates, 
          "stop times": stop_times, 
          "frequencies": frequencies, 
          "shapes": shapes, 
          "trips": trips, 
          "stops": stops, 
          "calendar": calendar, 
          "routes": routes}

access = f"Dict name: all_ds. \n\nKeys: {all_ds.keys()}, \nDataSets: {len(all_ds)}"
print(access)

Dict name: all_ds. 

Keys: dict_keys(['agency', 'dates', 'stop times', 'frequencies', 'shapes', 'trips', 'stops', 'calendar', 'routes']), 
DataSets: 9


In [5]:
# checking how prevalent missing values are in our data (for each dataset)

# defining a function to check null values:
def null_cols(ds):
    """check whether the value in each field is missing (null) and return either 
    True or False for each field, totaling up the number of True values by column. """
    return ds.isnull().sum()

# applying the filter to each dataset
agency_null_cols = null_cols(agency)
dates_null_cols = null_cols(dates)
stop_times_null_cols = null_cols(stop_times)
frequencies_null_cols = null_cols(frequencies)
shapes_null_cols = null_cols(shapes)
trips_null_cols = null_cols(trips)
stops_null_cols = null_cols(stops)
calendar_null_cols = null_cols(calendar)
routes_null_cols = null_cols(routes)

# Adding a condition that will filter the data and show us only columns where the number 
# of null values were greater than zero for each dataset:

# 'agency_phone' = 1
agency_null_cols[agency_null_cols > 0] 

# dates['exception_type'].value_counts() == 1 
dates_null_cols[dates_null_cols > 0]  

# ['stop_headsign', 'pickup_type', 'drop_off_type', shape_dist_traveled] = 1842 (All entries)
stop_times_null_cols[stop_times_null_cols > 0] 

# frequencies['exact_times'].value_counts() == 0
frequencies_null_cols[frequencies_null_cols > 0] 

# 'shape_dist_traveled' = 182 (All entries)
shapes_null_cols[shapes_null_cols > 0] 

# ['trip_headsign', 'direction_id', 'block_id'] = 132 (All entries)
trips_null_cols[trips_null_cols > 0]

# ['stop_code', 'stop_desc', 'zone_id', 'stop_url', 'location_type', 'parent_station'] = 49 (All entries)
stops_null_cols[stops_null_cols > 0]

# nothing to declare
calendar_null_cols[calendar_null_cols > 0]

# ['route_short_name', 'route_desc', 'route_url', 'route_color', 'route_text_color'] = 10 (All entries)
routes_null_cols[routes_null_cols > 0]

route_short_name    10
route_desc          10
route_url           10
route_color         10
route_text_color    10
dtype: int64

In [6]:
# Judgement call: droping information that we don't think it's going to be very useful 
# to our analysis (removing those columns from your datasets) with the drop method.
# We will add these column names to a list, and then we will pass those columns to the 
# drop method and indicate that we want columns (not rows) dropped by setting the axis 
# parameter to 1.

# defining a function to create a list:
def drop_cols(bad_cols):
    """Adding col names to a list to be droped; in this case as long as the col has a 
    single null value in it, since, in this case, if it has one, their all null."""
    return list(bad_cols[bad_cols > 0].index)

# TODO: Recheck dates, dates_drop_cols (and all other that have 'dates' in the name),
# as well as calendar, for I made a mistake. I rechecked it, but another pass would be good.

# applying the function to each ds
agency_drop_cols = drop_cols(agency_null_cols)
dates_drop_cols = drop_cols(dates_null_cols) # this one has no cols to drop
stop_times_drop_cols = drop_cols(stop_times_null_cols)
frequencies_drop_cols = drop_cols(frequencies_null_cols) # this one has no cols to drop
shapes_drop_cols = drop_cols(shapes_null_cols)
trips_drop_cols = drop_cols(trips_null_cols)
stops_drop_cols = drop_cols(stops_null_cols)
calendar_drop_cols = drop_cols(calendar_null_cols) # this one has no cols to drop
routes_drop_cols = drop_cols(routes_null_cols)

# Passing those columns to the drop method and indicate that we want columns (not rows) 
# dropped by setting the axis parameter to 1:
agency = agency.drop(agency_drop_cols, axis = 1)
dates = dates.drop(dates_drop_cols, axis = 1)
stop_times = stop_times.drop(stop_times_drop_cols, axis = 1)
frequencies = frequencies.drop(frequencies_drop_cols, axis = 1)
shapes = shapes.drop(shapes_drop_cols, axis = 1)
trips = trips.drop(trips_drop_cols, axis = 1)
stops = stops.drop(stops_drop_cols, axis = 1)
calendar = calendar.drop(calendar_drop_cols, axis = 1)
routes = routes.drop(routes_drop_cols, axis = 1)

# this should've left us w/no cols with null values in the datasets

In [7]:
print(access)

# checking Incorrect Values in our data (for each dataset)
# The dataset was uploaded at 01/01/2020

# all the dates in the 'start_date' pertain to 16/09/2019
calendar['start_date'].value_counts()

# all the dates in the 'end_date' pertain to 16/09/2029; what does this mean?
calendar['end_date'].value_counts()

# we have values for dates that are for dates in 2020 that haven't occured yet. Why?
# What does 'exception_type' stand for? And why are all entries in it == 1?
dates['date'].value_counts()
dates['exception_type'].value_counts()

# varies between 18 and 8: yellow == 13, green == 13, red == 12, blue == 18, so 
stop_times['trip_id'].value_counts()

# varies between 24 and 1
stop_times['arrival_time'].value_counts()
stop_times['departure_time'].value_counts()

# varies between 74 and 26
stop_times['stop_id'].value_counts()

# varies between 132 and 40
stop_times['stop_sequence'].value_counts()

# all 'trip_id's == 1
frequencies['trip_id'].value_counts()

# varies between 14 for 485 and 2 for several
frequencies['headway_secs'].value_counts()

# all == 0
frequencies['exact_times'].value_counts()

# between 18 and 8
shapes['shape_id'].value_counts()

# interesting... look at 'route_id' w/ 'shape_id'
[col and trips[col].value_counts() for col in trips]
trips[trips['shape_id'] == 167]
trips['shape_id'].value_counts()

# all entries are unique 
# (13+13+12+18-6, where -6 are intersections == 50; it has 49 entries? ARROIOS is missing
# from this list)
len(stops['stop_id'].unique())

# all (132) unique
calendar['service_id'].value_counts()

# we can drop 'agency_id' and 'route_type'


Dict name: all_ds. 

Keys: dict_keys(['agency', 'dates', 'stop times', 'frequencies', 'shapes', 'trips', 'stops', 'calendar', 'routes']), 
DataSets: 9


25816319    1
25816368    1
25816354    1
25816355    1
25816356    1
           ..
25816281    1
25816282    1
25816283    1
25816284    1
25816320    1
Name: service_id, Length: 132, dtype: int64

In [8]:
print(access)
# Identifying Low Variance Columns (where the majority of the values in the column are the 
# same). Since there is not a lot of variability in these columns, they have the potential 
# to not be as informative as columns that have a variety of different values in them.

def low_variance(ds):
    """Creating a function that iterates over each col, takes the minimum and the 90th 
    percentile value for all the numeric columns in our data set (identified via the 
    _get_numeric_data method), and if the 90th percentile and the minimum are equal to each 
    other, meaning that at least 90% of the values in that column are the same, it will 
    add that column name to a low_variance list."""
    
    return [col for col 
            in ds._get_numeric_data() 
            if min(ds[col]) == np.percentile(ds[col], 90)]

# Creating an empty list (low_variance) to hold the names of columns that fit our criteria:
agency_low_variance = low_variance(agency) # 'agency_id'
dates_low_variance = low_variance(dates) # 'exception_type'
stop_times_low_variance = low_variance(stop_times) # none
frequencies_low_variance = low_variance(frequencies) # 'exact_times'
shapes_low_variance = low_variance(shapes) # none
trips_low_variance = low_variance(trips) # none
stops_low_variance = low_variance(stops) # none
calendar_low_variance = low_variance(calendar) # ['start_date', 'end_date']
routes_low_variance = low_variance(routes) # ['agency_id', 'route_type']

# Checking the values that do exist in these fields to confirm that they are not very 
# informative. 
agency[agency_low_variance]
dates['exception_type'].value_counts()
frequencies['exact_times'].value_counts()
calendar['start_date'].value_counts()
calendar['end_date'].value_counts()
routes[routes_low_variance]

# Using the the drop method to remove those columns (axis = 1) from our data frame.
agency = agency.drop(agency_low_variance, axis = 1)
dates = dates.drop(dates_low_variance, axis = 1)
frequencies = frequencies.drop(frequencies_low_variance, axis = 1)
#ca

Dict name: all_ds. 

Keys: dict_keys(['agency', 'dates', 'stop times', 'frequencies', 'shapes', 'trips', 'stops', 'calendar', 'routes']), 
DataSets: 9


In [11]:
print(access)

# Extreme Values and Outliers
# These outliers can influence our aggregations when we are analyzing data later, so we 
# want to make sure we address them during our data cleaning stage.

# Using IQR (Inter Quartile Range) to identify outliers. 

# Once the IQR is calculated, it is multiplied by a constant (typically 1.5), and lower and 
# upper bounds are established at:
# 25th Percentile - (IQR x 1.5)
# 75th Percentile + (IQR x 1.5)

# Any values outside this range are potential outliers and should be investigated.
# We will use the Pandas describe function to easily calculate the 25th and 75th 
# percentiles for every column and transpose the results so that we can easily reference 
# the values in calculating the interquartile ranges.

def iqr(ds):
    """creates a IQR column from a dataset and transposes the results."""
    return ds.describe().transpose()

# Creating the IQR tale for each dataset by applying the iqr function to each dataset.
# It doesn't work (and it isn't necessary?) for agency
dates_stats = iqr(dates)
stop_times_stats = iqr(stop_times)
frequencies_stats = iqr(frequencies)
shapes_stats = iqr(shapes)
trips_stats = iqr(trips)
stops_stats = iqr(stops)
calendar_stats = iqr(calendar)
routes_stats = iqr(routes)

# Adding 'IQR' table to the stats of each dataset
dates_stats['IQR'] = dates_stats['75%'] - dates_stats['25%']
stop_times_stats['IQR'] = stop_times_stats['75%'] - stop_times_stats['25%']
frequencies_stats['IQR'] = frequencies_stats['75%'] - frequencies_stats['25%']
shapes_stats['IQR'] = shapes_stats['75%'] - shapes_stats['25%']
trips_stats['IQR'] = trips_stats['75%'] - trips_stats['25%']
stops_stats['IQR'] = stops_stats['75%'] - stops_stats['25%']
calendar_stats['IQR'] = calendar_stats['75%'] - calendar_stats['25%']
routes_stats['IQR'] = routes_stats['75%'] - routes_stats['25%']

# Create an empty data frame called outliers with the same columns as our data set. 
# Finally, we will loop through each column in the data calculating the lower and upper 
# bounds, retrieving records where the value for that column falls outside the bounds we 
# established, and appending those results to our outlier data frame.

outliers = pd.DataFrame(columns=dates.columns)

for col in dates_stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = data[(data[col] < lower) | 
                   (data[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)

#Our outliers data frame should now be populated with records that you can investigate further and determine whether they should be kept in the data or dropped. The Outlier column we added before appending the results for the column to the outliers data frame will let you know what column in each record contained the outlier. If you find that this method is returning too many results, you can be more stringent with your cutoff criteria (e.g. increasing the constant by which you multiply the IQR to 3 instead of 1.5).

outliers

Dict name: all_ds. 

Keys: dict_keys(['agency', 'dates', 'stop times', 'frequencies', 'shapes', 'trips', 'stops', 'calendar', 'routes']), 
DataSets: 9


NameError: name 'stats' is not defined

In [20]:
print(access)
routes_stats

Dict name: all_ds. 

Keys: dict_keys(['agency', 'dates', 'stop times', 'frequencies', 'shapes', 'trips', 'stops', 'calendar', 'routes']), 
DataSets: 9


,count,mean,std,min,25%,50%,75%,max,IQR
agency_id,10.0,2.0,0.000000,2.0,2.00,2.0,2.00,2.0,0.0
route_id,10.0,11275.3,21002.128083,164.0,166.25,170.5,4921.75,50954.0,4755.5
route_type,10.0,1.0,0.000000,1.0,1.00,1.0,1.00,1.0,0.0


In [ ]:
frequencies

In [ ]:
def sec_to_min(secs):
    minu = int(secs / 60)
    seconds = (secs % 60)
    #seconds = int(secs % 60)
    return f"{minu}:{seconds}"

#frequencies['headway_min'] = sec_to_min(frequencies['headway_secs'])

for elemento in frequencies['headway_secs']:
    
    row_in = sec_to_min(elemento)
   # print(row_in)
    
frequencies['headway_min'] = [sec_to_min(elemento) for elemento in frequencies['headway_secs']]
frequencies



In [ ]:
print(access)

# 164 e 4921
# 165 e 4922
shapes['shape_id'].unique()


In [230]:
# trip_id_total_time = int(end_time - start_time) in secs

"""def treat_trip_times(time):
    
    split = time.split(":", 2)[:3]
    
    converted_to_secs = (int(split[0]) * 60 * 60) + (int(split[1]) * 60) + (int(split[2]))
    
    return converted_to_secs


#treat_trip_times(frequencies['end_time'][1]) - treat_trip_times(frequencies['start_time'][1])
end = list(map(treat_trip_times, frequencies['end_time']))
start = list(map(treat_trip_times, frequencies['start_time']))

#frequencies['trip_total_time'] 

# type(treat_trip_times("10:00:00"))
end"""

[26100,
 34200,
 36000,
 60000,
 68400,
 70200,
 72000,
 75600,
 90300,
 43200,
 90300,
 43200,
 90300,
 3900,
 75600,
 72000,
 70200,
 68400,
 60000,
 36000,
 34200,
 26100,
 43200,
 3900,
 43200,
 3900,
 25200,
 27000,
 34200,
 36000,
 59400,
 70200,
 72000,
 73800,
 75600,
 77400,
 81000,
 90300,
 43200,
 73800,
 81000,
 90300,
 43200,
 73800,
 81000,
 90300,
 3900,
 81000,
 77400,
 75600,
 73800,
 72000,
 70200,
 59400,
 36000,
 34200,
 27000,
 25200,
 3900,
 81000,
 73800,
 43200,
 3900,
 81000,
 73800,
 43200,
 26100,
 35100,
 59400,
 70200,
 73800,
 81000,
 90300,
 43200,
 73800,
 80100,
 90300,
 43200,
 73800,
 80100,
 90300,
 3900,
 81000,
 73800,
 70200,
 59400,
 35100,
 26100,
 43200,
 80100,
 73800,
 3900,
 43200,
 80100,
 73800,
 3900,
 59400,
 59400,
 26100,
 36000,
 58500,
 73800,
 75600,
 80100,
 90900,
 43200,
 73800,
 77400,
 81000,
 90300,
 43200,
 73800,
 77400,
 81000,
 90300,
 4500,
 80100,
 75600,
 73800,
 58500,
 36000,
 26100,
 3900,
 81000,
 77400,
 73800,
 43

In [ ]:
# iterate start_time
# remove "00" after second ":"
# transform in int
# covert to seconds

# repeat the above proccess for end_time

# subtract start_time from end_time and assing to a new col called 'trip_id_total_time'



# df['DataFrame Column'] = pd.to_numeric(df['DataFrame Column'])
#frequencies['trip_id_total_time'] = 
"""frequencies['end_time'] = pd.to_numeric(frequencies['end_time'])
frequencies['start_time'] = pd.to_numeric(frequencies['start_time'])"""
#frequencies['trip_id_total_time']

#frequencies['trip_id_total_time'] = [(int(x[0]))*60*60 + int(x[1])*60 + int(x[2]) for x in l]
#frequencies

#[(int(x[0]))*60*60 + int(x[1])*60 + int(x[2]) for x in l]


   """ split_start = [trip_time.split(":", 2)[:2]
                   for trip_time 
                   in start]
    
    split_end = [trip_time.split(":", 2)[:2]
                   for trip_time 
                   in end]
    
    start_secs = [int(val) for sublist in split_start for val in sublist]
    
    end_secs = [int(val) for sublist in split_end for val in sublist]
    """
    """valor_querido = [(end_secs[val]*60*60 + end_secs[val]*60 + end_secs[val]) - 
                     (start_secs[val]*60*60 + start_secs[val]*60 + start_secs[val]) 
                     for sublist in start for val in sublist]"""
    
    #start_secs = [(int(split_start[i][i]) * 60 + int(split_start[i][j])) for i in split_start]

In [14]:
verde = [‘ALAMEDA’,
         ‘ALVALADE’,
         ‘ANJOS’,
         ‘AREEIRO’,
         ‘BAIXA-CHIADO’,
         ‘CAIS DO SODRE’,
         ‘CAMPO GRANDE’,
         ‘INTENDENTE’,
         ‘MARTIM MONIZ’,
         ‘ROMA’,
         ‘TELHEIRAS’]

azul = [‘ALFORNELOS’,
        ‘ALTO DOS MOINHOS’,
        ‘AMADORA ESTE’,
        ‘AVENIDA’,
        ‘BAIXA-CHIADO’,
        ‘CARNIDE’,
        ‘COLÉGIO MILITAR-LUZ’,
        ‘JARDIM ZOOLÓGICO’,
        ‘LARANJEIRAS’,
        ‘MARQUÊS DE POMBAL’,
        ‘PARQUE’,
        ‘PONTINHA’,
        ‘PRAÇA DE ESPANHA’,
        ‘REBOLEIRA’,
        ‘RESTAURADORES’,
        ‘ROSSIO’,
        ‘SANTA APOLÓNIA’,
        ‘SÃO SEBASTIÃO’,
        ‘TERREIRO DO PAÇO’]

vermelha = [‘AEROPORTO',
            ‘ALAMEDA’,
            ‘BELA VISTA’,
            ‘CABO RUIVO’,
            ‘CHELAS’,
            ‘ENCARNAÇÃO’,
            ‘MOSCAVIDE’,
            ‘OLAIAS’,
            ‘OLIVAIS’,
            ‘ORIENTE’,
            ‘SALDANHA’,
            ‘SÃO SEBASTIÃO’]

amarela = [‘AMEIXOEIRA’,
           ‘CAMPO GRANDE’,
           ‘CAMPO PEQUENO’,
           ‘CIDADE UNIVERSITÁRIA’,
           ‘ENTRECAMPOS’,
           ‘LUMIAR’,
           ‘MARQUÊS DE POMBAL’,
           ‘ODIVELAS’,
           ‘PICOAS’,
           ‘QUINTA DAS CONCHAS’,
           ‘RATO’,
           ‘SALDANHA’,
           ‘SR. ROUBADO’]

roxa = [‘CAMPO GRANDE’,
        ‘CAMPO PEQUENO’,
        ‘CIDADE UNIVERSITÁRIA’,
        ‘ENTRECAMPOS’,
        ‘MARQUÊS DE POMBAL’,
        ‘PICOAS’,
        ‘RATO’,
        ‘SALDANHA’]

SyntaxError: invalid character in identifier (<ipython-input-14-062edadd95e6>, line 1)